In [5]:
import os
import tarfile
import urllib
import shutil
import json
import random
import numpy as np
import boto3
import sagemaker

from tqdm import tqdm
from sagemaker.amazon.amazon_estimator import get_image_uri
from matplotlib import pyplot as plt

role = sagemaker.get_execution_role()
print(role)

sess = sagemaker.Session()
print(sess)

arn:aws:iam::108782086480:role/service-role/AmazonSageMaker-ExecutionRole-20241028T231313


# Sagemaker data preparation

In [ ]:
sess = sagemaker.Session()

s3_train_path = sess.upload_data(path='train', bucket=bucket_name, key_prefix='train')
s3_train_lst_path = sess.upload_data(path='train_lst', bucket=bucket_name, key_prefix='train_lst')
s3_validation_path = sess.upload_data(path='validation', bucket=bucket_name, key_prefix='validation')
s3_validation_lst_path = sess.upload_data(path='validation_lst', bucket=bucket_name, key_prefix='validation_lst')
print(s3_train_path, s3_train_lst_path, s3_validation_path, s3_validation_lst_path)


# Sagemaker Estimator

In [6]:
model = sagemaker.estimator.Estimator(
    image_uri="108782086480.dkr.ecr.us-east-1.amazonaws.com/simple-docker-service-0e4ed6c4db93:latest",
    role=role,
    instance_count=1,
    instance_type='ml.t3.medium',
    volume_size=5,
    train_max_run=10,
    input_mode='File',
    output_path='s3://mnist-20241029/output',
    sagemaker_session=sess
)

train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
model.set_hyperparameters(
    num_layers=18,
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=37,
    mini_batch_size=32,
    resize=224,
    epochs=10,
    learning_rate=0.001,
    num_training_samples=train_count,
    use_weighted_loss=1,
    augmentation_type='crop_color_transform',
)

In [ ]:
data_channels = { 
    'train': 'train_data', 
    'validation': 'validation_data' 
}

In [ ]:
model.fit(inputs=data_channels, logs=True)

# model deploy

In [ ]:
predictor = model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    image_uri='your-custom-image-uri'
)

In [ ]:
# sagemaker.Session().delete_endpoint(predictor.endpoint)